1\. Write a function that converts number representation (bin<->dec<->hex)

In [18]:
def rappr_to_10(stringa, base):
    num = 0
    power = len(stringa) - 1
    for c in stringa:
        if c == "A":
            c = 10
        if c == "B":
            c = 11
        if c == "C":
            c = 12
        if c == "D":
            c = 13
        if c == "E":
            c = 14
        if c == "F":
            c = 15

        if int(c) >= base:
            return "The number is not valid in this representation"

        num += base**power * int(c)
        power -= 1
    return str(num)

def convert_from_10(stringa, base):
    num = int(stringa)
    l = ""
    while num >= base:
        resto = num%base
        num = int((num - resto)/base)
        if resto == 10:
            resto = "A"
        if resto == 11:
            resto = "B"
        if resto == 12:
            resto = "C"
        if resto == 13:
            resto = "D"
        if resto == 14:
            resto = "E"
        if resto == 15:
            resto = "F"
        l = str(resto) + l


    resto = num%base

    if resto == 10:
         resto = "A"
    if resto == 11:
        resto = "B"
    if resto == 12:
        resto = "C"
    if resto == 13:
        resto = "D"
    if resto == 14:
        resto = "E"
    if resto == 15:
        resto = "F"

    l = str(resto) + l

    return l


print(convert_from_10("163", 5))

print(rappr_to_10(convert_from_10("163", 5), 5))

1123
163


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [22]:
"""
# PROVE VARIE
print(rappr_to_10("11111111111111111111111111111111", 2))
max_ = 2**32-1
# max = 4294967295
print(max_)
"""

def double_num(num_ins):
    mask_sign = int(rappr_to_10("10000000000000000000000000000000", 2))
    mask_exp  = int(rappr_to_10("01111111100000000000000000000000", 2))
    mask_mant = int(rappr_to_10("00000000011111111111111111111111", 2))

    # print(mask_sign, '\t', mask_exp, '\t', mask_mant)

    # num_prova = int(3333333333)    # Testing a number with sign -
    num_prova = int(num_ins)

    sign_ = num_prova & mask_sign
    exp_  = num_prova & mask_exp
    mant_ = num_prova & mask_mant

    # TEST VARI
    # print(sign_, '\t', exp_, '\t', mant_)
    # convert_from_10(12345678, 2)

    sign_ >>= 31        # Moving sign_ to the first bit, where it will be 0 or 1
    # print(sign_)      # Testing sign_
    exp_ >>= 23         # Moving exp to the firsts 8 bits, so I can work with it
    exp_ -= (127)       # Adding bias
    # print(exp_)
    # mant_ >>= 0       # NO NEED TO SHIFT mant_

    # N.B.: NEED TO DIVIDE FOR 2^(-23) TO HAVE THE CORRECT MANTISS
    num_ = ((-1)**sign_) * ( mant_*(2**(exp_-23)) + 2**(exp_) )

    return(num_)

print(double_num(3333333333))

-22352.666015625


In [76]:
# CELLA ERRATA!!! (non più modificata...)

# use masks
# use effective bits, not strings... (int from 0 to 2147483647 (which is 2**31 - 1))

prova = "00000000000000000000000000000001"
s =     "00000000000000000000000000000001"
expo =  "00000000000000000000000111111110"
mant =  "11111111111111111111111000000000"

def and_str(stringa, mask):
    app = ""
    if len(stringa) != len(mask):
        return "Strings ha not the same langht"
    for c1, c2 in zip(stringa, mask):
        app = app + str(int(c1)*int(c2))        
    return app


nums = and_str(prova, s)

print(nums)

numexpo = and_str(prova, expo)

print(numexpo)

nummant = and_str(prova, mant)

print(nummant)


expo_bin = ""

for c in range (23, 32):
    expo_bin += numexpo[c]

expo_ = int(rappr_to_10(expo_bin, 2))

print(expo_)

num_str = ""

for c in range(0, 23):
    num_str += nummant[c]

    
num_ = 0
for c in range(0, 23):
    num_ = int(rappr_to_10((num_str), 2)) / (2**23)

num_ += 2**(expo_+1)


if int(nums[31]) == 1:
    num_ *= -1

print(num_)


00000000000000000000000000000001
00000000000000000000000000000000
00000000000000000000000000000000
0
-2.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [8]:
uf = 1
cont_uf = 0
of = float(1)
cont_of = 0

while uf != 0:
    uf /= 2
    cont_uf += 1
    
while of < 2*of:
    app = of
    of *= 2
    cont_of += 1

print("Overflow doubled ", cont_of, " times.")

print(app)    # N.B.: app is not the maximum value...

print("Underflow halved ", cont_uf, " times.")

print("Underflow: ", 2**-(cont_uf-1))

Overflow doubled  1024  times.
8.98846567431158e+307
Underflow halved  1075  times.
Underflow:  5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [21]:
num = 1.
eps = 1.

while num != num + eps:
    eps /= 2

eps *= 2    # Serve il penultimo epsilon, poiché l'ultimo è quello che diventa irrilevante

print(eps)

2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [28]:
def solut_square(a, b, c):
    x1 = (-b + pow(b**2 - 4*a*c , 0.5))/(2*a)
    x2 = (-b - pow(b**2 - 4*a*c , 0.5))/(2*a)
    solut = [x1, x2]
    return solut

a_test = 0.001
b_test = 1000
c_test = 0.001

sols = solut_square(a_test, b_test, c_test)

print(sols)

def solut_alternative(a, b, c):
    x1 = 4*a*c/(2*a*(-b - pow(b**2 - 4*a*c , 0.5)))
    x2 = 4*a*c/(2*a*(-b + pow(b**2 - 4*a*c , 0.5)))
    solut = [x1, x2]
    return solut

sols_altern = solut_alternative(a_test, b_test, c_test)
print(sols_altern)

def solut_correct(a, b, c):
    if b>0:
        x2 = (-b - pow(b**2 - 4*a*c , 0.5))/(2*a)
        x1 = 4*a*c/(2*a*(-b - pow(b**2 - 4*a*c , 0.5)))
        sols = [x1, x2]
    else:
        x1 = (-b + pow(b**2 - 4*a*c , 0.5))/(2*a)
        x2 = 4*a*c/(2*a*(-b + pow(b**2 - 4*a*c , 0.5)))
        sols = [x1, x2]
    return sols


sols_correct = solut_correct(a_test, b_test, c_test)
print(sols_correct)
sols_correct = solut_correct(a_test, -b_test, c_test)    # Test con b<0
print(sols_correct)

[-9.999894245993346e-07, -999999.999999]
[-1.0000000000009999e-06, -1000010.5755125057]
[-1.0000000000009999e-06, -999999.999999]
[999999.999999, 1.0000000000009999e-06]


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

(a) The true anwer is:

$ \frac{{\rm d}f}{{\rm d}x} \big\vert_{x=1} = \frac{{\rm d}}{{\rm d}x} (x^2 - x) \big\vert_{x=1} = 2x - 1 \big\vert_{x=1} = 1 $

In [20]:
def f(x):
    return x*(x-1)

x = 1
δ = 1e-2
der = (f(x+δ) - f(x)) / δ

print(der, "\n")

for exp in range(2, 15, 2):
    δ = 10**(-exp)
    der = (f(x+δ) - f(x)) / δ
    print("δ = ", δ, ". Il risultato è: ", der, ", con un'accuratezza di: ", 1-der)

print("\n" + "Ora provo con un passo diverso (pura intuizione, voglio solo provare...) \n")

for exp in range(2, 60, 2):
    δ = 2**(-exp)
    der = (f(x+δ) - f(x)) / δ
    der_better = f(x+δ)/δ - f(x)/δ
    print("δ = ", δ, ". Il risultato è: ", der, ", con un'accuratezza di: ", 1-der)
    print("δ = ", δ, ". Il risultato alternativo è: ", der_better, ", con un'accuratezza di: ", 1-der)

1.010000000000001 

δ =  0.01 . Il risultato è:  1.010000000000001 , con un'accuratezza di:  -0.010000000000000897
δ =  0.0001 . Il risultato è:  1.0000999999998899 , con un'accuratezza di:  -9.999999988985486e-05
δ =  1e-06 . Il risultato è:  1.0000009999177333 , con un'accuratezza di:  -9.99917733279787e-07
δ =  1e-08 . Il risultato è:  1.0000000039225287 , con un'accuratezza di:  -3.922528746258536e-09
δ =  1e-10 . Il risultato è:  1.000000082840371 , con un'accuratezza di:  -8.284037100736441e-08
δ =  1e-12 . Il risultato è:  1.0000889005833413 , con un'accuratezza di:  -8.890058334132256e-05
δ =  1e-14 . Il risultato è:  0.9992007221626509 , con un'accuratezza di:  0.0007992778373491216

Ora provo con un passo diverso (pura intuizione, voglio solo provare...) 

δ =  0.25 . Il risultato è:  1.25 , con un'accuratezza di:  -0.25
δ =  0.25 . Il risultato alternativo è:  1.25 , con un'accuratezza di:  -0.25
δ =  0.0625 . Il risultato è:  1.0625 , con un'accuratezza di:  -0.0625
δ =  0.

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt{(1-x^2)} {\rm d}x = 2 \int_{0}^{1} \sqrt{(1-x^2)} {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute?

Answer to (b)

It depends on the computer...


In [58]:
def f_rad(x):
    return pow(1-x*x, 0.5)

N = 100000000
h = 2/N

I = 0

def process(n):
    I = 0
    for i in range(n):
        I += h*f_rad(i*h-1.)
    return I

%timeit process(N)

I = process(N)

print(I)

41.4 s ± 34.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.570796326793284
